# Part 4 - RAG Basics: Teaching AI About Your Documents

Build RAG systems to answer questions from your documents.

In [ ]:
!pip install -q langchain langchain-openai langchain-community faiss-cpu

In [ ]:
import os
import getpass

if not os.getenv('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key: ')

## Step 1: Load & Split Documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# Sample document
with open('sample_doc.txt', 'w') as f:
    f.write('''Our vacation policy: Employees get 15 vacation days per year.
Sick leave: Unlimited sick days with doctor note.
Remote work: 2 days per week allowed.''')

loader = TextLoader('sample_doc.txt')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)
print(f'Split into {len(chunks)} chunks')

## Step 2: Create Vector Store

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
print('Vector store created!')

## Step 3: Build RAG Chain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = vectorstore.as_retriever()

template = '''Answer based on context:

Context: {context}

Question: {question}'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

answer = rag_chain.invoke('How many vacation days do employees get?')
print(answer)